In [1]:
import talib as ta
from datetime import datetime
from datetime import timedelta
from bigtrader.constant import Direction, OrderType

STRATEGY_NAME = "STRATEGY_1m"
        
def initialize(context):
    """初始化"""
    print("initialize")  
#     context.symbol = instruments
    context.my_instruments = context.get_conf_param("instruments")#从传入参数中获取需要交易的合约
    context.closetime_day = context.get_conf_param("closetime_day")#日内策略白盘平仓时间，一般14:58
    context.closetime_night = context.get_conf_param("closetime_night")#日内策略夜盘平仓时间，一般22:58，注意有些商品夜盘收盘时间不一样
    context.order_num = context.get_conf_param("order_num")#下单手数
    context.set_universe(context.my_instruments)#设置需要处理的合约
    context.N = 5
    context.k1 = 0.2
    context.k2 = 0.2

def before_trading(context, data):
    """盘前处理"""
    #print("before_trading")
    context.subscribe(context.my_instruments) #注册合约
    context.index = 0
    context.flag = 1 #用于获取今开
    # 取历史数据
    history_data_for_thrust = data.history(context.my_instruments, ["open","high","low","close"], context.N, "1d")
    # 计算Dual Thrust 的上下轨
    HH = history_data_for_thrust['high'].max()
    HC = history_data_for_thrust['close'].max()
    LC = history_data_for_thrust['close'].min()
    LL = history_data_for_thrust['low'].min()
    context.range = max(HH - LC, HC - LL)
#     for instr in context.my_instruments:
#         index = instr.find('.')
#         #获取当前主力合约，如果含有.即指定某个具体合约，否则认为是需要处理主连合约
#         if index<=0:
#             context.current_instruments.append(data.current_dominant(instr))
#         else:#使用指定合约
#             context.current_instruments.append(instr)                                   
#     print("当前主力合约", context.current_instruments)
    
def handle_data(context, data):
    """Bar行情推送"""
    #获取开盘时间
    if context.flag == 1 :
        context.history_data = data.history(context.my_instruments, ["open"], 1, "1m")
        context.flag = context.flag + 1
        context.today_open = context.history_data.iloc[-1]['open'] 
    cur_date = data.current_dt
    cur_hm = cur_date.strftime('%H:%M') #time
    current_open = context.today_open
    context.buy_line = context.today_open + context.range * context.k1  # 上轨
    context.sell_line = context.today_open - context.range * context.k2  # 下轨
    # 分别获取多头持仓，和空头持仓
    position_long = context.get_position(context.my_instruments, Direction.LONG)
    position_short = context.get_position(context.my_instruments, Direction.SHORT)
    # 获取当前价格
    price = data.current(context.my_instruments, "close")    
    #尾盘平仓
    #部分品种夜盘收盘时间不一样，此时间表示指定的尾盘平仓时间往后偏移30分钟，这段时间内不能开新仓，只能平仓。给30分钟是为了足够的冗余
    closetime_nightshift = (datetime.strptime(context.closetime_night,'%H:%M') + timedelta(minutes = 30)).strftime('%H:%M')
    if((cur_hm>=context.closetime_day and cur_hm<="15:00") or (cur_hm>=context.closetime_night and cur_hm<=closetime_nightshift)):
        if(position_long.current_qty != 0):
            rv = context.sell_close(context.my_instruments, position_long.avail_qty, price, order_type=OrderType.MARKET)
            msg = str(data.current_dt) +  " 尾盘平多 for " + context.my_instruments + " 最新价=" + str(price) + " rv:" + str(rv)
            context.write_log(msg, stdout=1) 
        if(position_short.current_qty != 0):
            rv = context.buy_close(context.my_instruments, position_short.avail_qty, price, order_type=OrderType.MARKET)
            msg = str(data.current_dt) +  " 尾盘平空 for " + context.my_instruments + " 最新价=" + str(price) + " rv:" + str(rv)
            context.write_log(msg, stdout=1) 
        #尾盘不开新仓，直接返回
        return
    #交易逻辑
    if price > context.buy_line:
        if position_long.current_qty != 0:
            return
        elif position_short.current_qty != 0:
            rv = context.buy_close(context.my_instruments, position_short.avail_qty, price, order_type=OrderType.MARKET)
            msg = str(data.current_dt) +  " 平空 for " + context.my_instruments + " 最新价=" + str(price) + " rv:" + str(rv)
            context.write_log(msg, stdout=1)
            rv = context.buy_open(context.my_instruments, context.order_num, price, order_type=OrderType.MARKET)
            msg = str(data.current_dt) +  " 开多 for " + context.my_instruments + " 最新价=" + str(price) + " rv:" + str(rv)
            context.write_log(msg, stdout=1) 
        else:
            rv = context.buy_open(context.my_instruments, context.order_num, price, order_type=OrderType.MARKET)
            msg = str(data.current_dt) +  " 开多 for " + context.my_instruments + " 最新价=" + str(price) + " rv:" + str(rv)
            context.write_log(msg, stdout=1)     
    elif price < context.sell_line:
        if position_short.current_qty != 0:
            return
        elif position_long.current_qty != 0:
            rv = context.sell_close(context.my_instruments, position_long.avail_qty, price, order_type=OrderType.MARKET)
            msg = str(data.current_dt) +  " 平多 for " + context.my_instruments + " 最新价=" + str(price) + " rv:" + str(rv)
            context.write_log(msg, stdout=1) 
            rv = context.sell_open(context.my_instruments, context.order_num, price, order_type=OrderType.MARKET)
            msg = str(data.current_dt) +  " 开空 for " + context.my_instruments + " 最新价=" + str(price) + " rv:" + str(rv)
            context.write_log(msg, stdout=1)    
        else:
            rv = context.sell_open(context.my_instruments, context.order_num, price, order_type=OrderType.MARKET)
            msg = str(data.current_dt) +  " 开空 for " + context.my_instruments + " 最新价=" + str(price) + " rv:" + str(rv)
            context.write_log(msg, stdout=1)  

            
def handle_order(context, order):
    """委托回报推送"""
    msg = "handle_order:" + order.log_str()
    context.write_log(msg, stdout=1) 
         
def handle_trade(context, trade):
    """成交回报推送"""
    msg = "handle_trade:" + trade.log_str()
    context.write_log(msg, stdout=1) 
    # 分别获取多头持仓，和空头持仓
    position_long = context.get_position(trade.symbol, Direction.LONG)
    position_short = context.get_position(trade.symbol, Direction.SHORT)
    msg = "当前多头持仓:"+ str(position_long) + "当前空头持仓:"+ str(position_short)
    context.write_log(msg, stdout=1) 
    

instruments = "RB2110.SHF" #["RB2110.SHF"] #["RB2110.SHF"] [I2105.DCE"] ['RB']# RB2101.SHF 如果想用主连合约，可以出入合约简码，例如RB,I,CF

#需要交易者传入的参数
strategy_setting = [
    {
        "instruments": instruments,
        "order_num": 2,
        "closetime_day": "14:58",
        "closetime_night": "22:58"
    }    
    
]
start_date = "2021-04-21"
end_date = "2021-04-27"
md = M.hfbacktest.v1(start_date=start_date,
                     end_date=end_date,
                     instruments=[instruments], #只传入一个合约便于策略逻辑展示
                     capital_base=100000,
                     product_type=Product.FUTURE,
                     frequency=Frequency.MINUTE,
                     initialize=initialize,
                     before_trading_start=before_trading,
                     handle_data=handle_data,
                     handle_order=handle_order,
                     handle_trade=handle_trade,
                     plot_charts=True,
                     volume_limit=1.0,
                     disable_cache=0,
                     show_debug_info=1,
                     strategy_setting=strategy_setting,
                     slippage_type=SlippageType.FIXED,#滑点固定模式
                     slippage_value=1.0,#买卖双向各1个滑点
                     m_deps=np.random.rand())

[2021-05-21 13:54:02.545793] INFO: moduleinvoker: hfbacktest.v1 开始运行..

[2021-05-21 13:54:02.570978] INFO: hfbacktest: passed-in daily_data_ds:None

[2021-05-21 13:54:02.572886] INFO: hfbacktest: passed-in minute_data_ds:None

[2021-05-21 13:54:02.574588] INFO: hfbacktest: passed-in tick_data_ds:None

[2021-05-21 13:54:02.575673] INFO: hfbacktest: passed-in each_data_ds:None

...

2021-05-21 13:54:04.085385 strategy_20210521(bkt999,): handle_order:[bkt999,7,RB2110.SHF,long,open,0,2,5258.0,pending,11:26:00,7] 
2021-05-21 13:54:04.086278 strategy_20210521(bkt999,): 2021-04-23 11:26:00 开多 for RB2110.SHF 最新价=5258.0 rv:0 
2021-05-21 13:54:04.089356 strategy_20210521(bkt999,): handle_order:[bkt999,7,RB2110.SHF,long,open,2,2,5258.0,filled,11:26:00,7] 
2021-05-21 13:54:04.090179 strategy_20210521(bkt999,): handle_trade:[bkt999,7,RB2110.SHF,long,open,2,5259.0,SHFE.7,11:27:00.000000] 
2021-05-21 13:54:04.090376 strategy_20210521(bkt999,): 当前多头持仓:Position(bkt999,RB2110.SHF,long,current_qty:2,avail_qty:2,cost_price:5259.0,last_price:5259.0)当前空头持仓:Position(bkt999,RB2110.SHF,short,current_qty:0,avail_qty:0,cost_price:0.0,last_price:0.0) 
2021-05-21 13:54:04.159222 strategy_20210521(bkt999,): handle_order:[bkt999,8,RB2110.SHF,short,close_today,0,2,5291.0,pending,14:58:00,8] 
2021-05-21 13:54:04.159621 strategy_20210521(bkt999,): 2021-04-23 14:58:00 尾盘平多 for RB2110.SHF 最新价=5291

2021-05-21 13:54:04.426476 strategy_20210521(bkt999,): handle_order:[bkt999,12,RB2110.SHF,short,close_today,0,2,5380.0,pending,14:58:00,12] 
2021-05-21 13:54:04.427507 strategy_20210521(bkt999,): 2021-04-26 14:58:00 尾盘平多 for RB2110.SHF 最新价=5380.0 rv:0 
2021-05-21 13:54:04.431178 strategy_20210521(bkt999,): handle_order:[bkt999,12,RB2110.SHF,short,close_today,2,2,5380.0,filled,14:58:00,12] 
2021-05-21 13:54:04.432349 strategy_20210521(bkt999,): handle_trade:[bkt999,12,RB2110.SHF,short,close_today,2,5379.0,SHFE.12,14:59:00.000000] 
2021-05-21 13:54:04.432543 strategy_20210521(bkt999,): 当前多头持仓:Position(bkt999,RB2110.SHF,long,current_qty:0,avail_qty:0,cost_price:5416.0,last_price:5379.0)当前空头持仓:Position(bkt999,RB2110.SHF,short,current_qty:0,avail_qty:0,cost_price:0.0,last_price:0.0) 


2021-05-21 13:54:04.754621 run_backtest() finished! time cost 1.463s! 


[2021-05-21 13:54:05.822868] INFO: hfbacktest: backtest done, raw_perf_ds:DataSource(063a4b7bcf4d4216b10a4647ebdaf7b7T)

[2021-05-21 13:54:06.307236] INFO: moduleinvoker: hfbacktest.v1 运行完成[3.761489s].